In [2]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.26.1 which is incompatible.


In [8]:
from ortools.linear_solver import pywraplp

def university_scheduling():

    solver = pywraplp.Solver.CreateSolver('CBC')

    C = range(3)  # Ensemble des cours
    P = range(4)  # Ensemble des périodes
    S = range(2)  # Ensemble des salles
    E = range(2)  # Ensemble des enseignants
    G = range(2)  # Ensemble des groupes
    M = ['présentiel', 'enligne']  # Modalités d’enseignement

    d_c = [2, 1, 3]  # Durée de chaque cours
    n_g = [30, 25]  # Taille de chaque groupe
    q_s = [50, 40]  # Capacité de chaque salle
    delta_c_e = [[1, 0],  #Matrice dim(c,e) delta_0_1 = 0 : cours 0, prof 1 indisponible
                [1, 1],
                [0, 1]]
    alpha_g = [1, 0]  # Présentiel requis pour certains groupes

    # Variables de décision
    x = {}  # x[c, p, s, e] = 1 si le cours c est assigné à période p, salle s, enseignant e
    y = {}  # y[g, c, m] = 1 si le groupe g assiste au cours c avec la modalité m

    for c in C:
        for p in P:
            for s in S:
                for e in E:
                    x[c, p, s, e] = solver.IntVar(0, 1, f'x[{c},{p},{s},{e}]')

    for g in G:
        for c in C:
            for m in M:
                y[g, c, m] = solver.IntVar(0, 1, f'y[{g},{c},{m}]')

    # Contraintes
    # 1. Disponibilité des enseignants
    for c in C:
        for e in E:
            if delta_c_e[c][e] == 0:  # L'enseignant n'est pas disponible pour ce cours
                for p in P:
                    for s in S:
                        solver.Add(x[c, p, s, e] == 0)

    # 2. Pas de conflits pour les enseignants
    for p in P:
        for e in E:
            solver.Add(sum(x[c, p, s, e] for c in C for s in S) <= 1) #𝑒 ne peut pas enseigner plus d’un cours dans la période 𝑝 ∀ s.

    # 3. Capacité des salles
    for c in C:
        for s in S:
            for p in P:
                solver.Add(sum(n_g[g] * y[g, c, 'présentiel'] for g in G) <= q_s[s]) #nbre etudiants groupe g <= capacité salle s

    # 4. Modalité unique par groupe
    for g in G:
        for c in C:
            solver.Add(sum(y[g, c, m] for m in M) == 1) #Sum (nb_total modalités) choisies par le groupe g dans cours c doit être égale à 1

    # 5. Planification des salles
    for p in P: #applique la contrainte à p
        for s in S: ##applique la contrainte à s
            solver.Add(sum(x[c, p, s, e] for c in C for e in E) <= 1) #∀ c,e Sum (nb_total cours c dans salle s à période p <= 1)

    # 6. Durée des cours
    for c in C:
        solver.Add(sum(x[c, p, s, e] for p in P for s in S for e in E) == d_c[c]) #Impose que Sum(périodes) assignées au cours c == duree requise (x[0,p,s,e]=2)

    # 7. Exigence de présentiel
    for g in G:
        for c in C:
            if alpha_g[g] == 1:  # Si le groupe doit être en présentiel
                solver.Add(y[g, c, 'présentiel'] == 1)


    # Fonction objectif : minimiser les conflits et équilibrer les ressources
    solver.Minimize(
        sum(x[c, p, s, e] for c in C for p in P for s in S for e in E) +
        sum(y[g, c, m] for g in G for c in C for m in M)
    )

    # Résolution
    status = solver.Solve()

    # Résultats
    if status == pywraplp.Solver.OPTIMAL:
        print("Solution optimale trouvée :")
        for c in C:
            for p in P:
                for s in S:
                    for e in E:
                        if x[c, p, s, e].solution_value() > 0:
                            print(f"Cours {c} assigné à période {p}, salle {s}, enseignant {e}")
        for g in G:
            for c in C:
                for m in M:
                    if y[g, c, m].solution_value() > 0:
                        print(f"Groupe {g} suit le cours {c} en modalité {m}")
    else:
        print("Pas de solution optimale trouvée.")

# Appeler la fonction
university_scheduling()


Solution optimale trouvée :
Cours 0 assigné à période 0, salle 1, enseignant 0
Cours 0 assigné à période 3, salle 0, enseignant 0
Cours 1 assigné à période 2, salle 0, enseignant 0
Cours 2 assigné à période 0, salle 0, enseignant 1
Cours 2 assigné à période 1, salle 1, enseignant 1
Cours 2 assigné à période 3, salle 1, enseignant 1
Groupe 0 suit le cours 0 en modalité présentiel
Groupe 0 suit le cours 1 en modalité présentiel
Groupe 0 suit le cours 2 en modalité présentiel
Groupe 1 suit le cours 0 en modalité enligne
Groupe 1 suit le cours 1 en modalité enligne
Groupe 1 suit le cours 2 en modalité enligne


#Avancé

In [42]:
from ortools.linear_solver import pywraplp

def university_scheduling(group_names,   # ex: ["DIA1", "DIA2"]
                          rooms,         # ex: ["A", "B", "C", "D"]
                          subjects,
                          teachers,
                          teacher_specialties,
                          course_type,    # ex: {"Maths CM":"CM", "Maths TD":"TD", ...}
                          periods_per_day=4,
                          days_per_week=5):
    """
    Planification de CM (en ligne) et TD (en présentiel)
    pour des groupes (group_names) et des salles (rooms).
    Affichage final : on liste, pour chaque groupe, chaque créneau occupé.
    """

    # --- 1) Définition des ensembles ---
    # C: ensemble des cours
    C = range(len(subjects))
    # P: ensemble des périodes (5 jours × 4 créneaux = 20, par défaut)
    P = range(days_per_week * periods_per_day)
    # R: ensemble des salles
    R = range(len(rooms))
    # E: ensemble des enseignants
    E = range(len(teachers))
    # G: ensemble des groupes
    G = range(len(group_names))
    M = ['présentiel', 'enligne']

    # --- 2) Paramètres de base ---
    d_c = [2] * len(subjects)   # Durée (nombre de créneaux) de chaque cours
    n_g = [30] * len(group_names)  # 30 étudiants par groupe
    q_r = [60] * len(rooms)        # capacité de chaque salle

    solver = pywraplp.Solver.CreateSolver('CBC')
    if not solver:
        print("Erreur : le solveur n'a pas pu être initialisé.")
        return None

    # --- 3) Variables de décision ---
    # x[c, p, r, e] = 1 si le cours c est planifié
    #                 à la période p, salle r, enseignant e
    x = {}
    for c in C:
        for p in P:
            for r in R:
                for e in E:
                    x[c, p, r, e] = solver.IntVar(0, 1, f'x[{c},{p},{r},{e}]')

    # y[g, c, m] = 1 si le groupe g suit le cours c avec la modalité m
    # (Ici, on force TD=>présentiel, CM=>en ligne, via des contraintes)
    y = {}
    for g in G:
        for c in C:
            for m in M:
                y[g, c, m] = solver.IntVar(0, 1, f'y[{g},{c},{m}]')

    # --- 4) Contraintes ---

    # (A) Forcer la modalité (CM => en ligne, TD => présentiel)
    for c in C:
        subj = subjects[c]
        if course_type[subj] == "CM":
            for g in G:
                solver.Add(y[g, c, 'enligne'] == 1)
                solver.Add(y[g, c, 'présentiel'] == 0)
        elif course_type[subj] == "TD":
            for g in G:
                solver.Add(y[g, c, 'présentiel'] == 1)
                solver.Add(y[g, c, 'enligne'] == 0)

    # (B) Un enseignant ne peut enseigner que ses spécialités
    for c in C:
        for e in E:
            if subjects[c] not in teacher_specialties.get(teachers[e], []):
                for p in P:
                    for r in R:
                        solver.Add(x[c, p, r, e] == 0)

    # (C) Pas de conflit enseignant : un enseignant e ne peut donner deux cours en même temps
    for p in P:
        for e in E:
            solver.Add(
                sum(x[c, p, r, e] for c in C for r in R) <= 1
            )

    # (D) Capacité des salles : si un cours c est présentiel,
    #     la somme n_g[g] ne doit pas dépasser q_r[r].
    for c in C:
        for r in R:
            for p in P:
                # Dans ce modèle simplifié, on ne lie pas x et y par un big-M,
                # mais on impose seulement la somme si c'est "présentiel".
                solver.Add(
                    sum(n_g[g] * y[g, c, 'présentiel'] for g in G) <= q_r[r]
                )

    # (E) Occupation salle : dans un créneau p et une salle r, max 1 cours
    for p in P:
        for r in R:
            solver.Add(
                sum(x[c, p, r, e] for c in C for e in E) <= 1
            )

    # (F) Durée des cours : chaque cours c occupe d_c[c] créneaux (au total)
    for c in C:
        solver.Add(
            sum(x[c, p, r, e] for p in P for r in R for e in E) == d_c[c]
        )

    # --- 5) Objectif ---
    # Minimiser la somme de x
    solver.Minimize(
        sum(x[c, p, r, e] for c in C for p in P for r in R for e in E)
    )

    # --- 6) Résolution ---
    status = solver.Solve()

    # --- 7) Lecture du résultat ---
    if status == pywraplp.Solver.OPTIMAL:
        print("Solution optimale trouvée :\n")

        # Préparons une structure de sortie
        # Au final, on veut un "planning" pour chaque groupe g
        # => listant (cours, salle, enseignant, période, modalité).
        full_schedule = []

        for c in C:
            subj = subjects[c]
            # On sait si c'est un CM ou un TD
            is_cm = (course_type[subj] == "CM")

            # Récupérons tous les créneaux (p), salles (r), enseignants (e) où x=1
            # (Un cours peut être réparti sur d_c[c] créneaux, qui peuvent être distincts)
            assigned_slots = []
            for p in P:
                for r in R:
                    for e in E:
                        if x[c, p, r, e].solution_value() > 0:
                            assigned_slots.append((p, r, e))

            # Pour chaque groupe g, on sait si la modalité est présentiel ou en ligne
            # => y[g,c,'présentiel'] = 1 ou y[g,c,'enligne'] = 1
            for g in G:
                group_mod = 'enligne' if is_cm else 'présentiel'
                # (Avec la contrainte imposée, on n'a pas vraiment le choix :
                #  CM => enligne, TD => présentiel.)

                # Pour chaque créneau réellement affecté dans assigned_slots,
                # on fabrique une entrée "g, c, p, r, e, modality"
                for (p, r, e) in assigned_slots:
                    if is_cm:
                        # En ligne => pas de salle
                        salle_label = "Aucune salle (en ligne)"
                    else:
                        # Présentiel => la salle r
                        salle_label = rooms[r]

                    full_schedule.append({
                        "group": group_names[g],
                        "subject": subj,
                        "teacher": teachers[e],
                        "period": f"Période {p + 1}",
                        "room": salle_label,
                        "modality": "enligne" if is_cm else "présentiel"
                    })

        # Tri optionnel pour un affichage "logique"
        # (ex: par group, puis par period)
        full_schedule.sort(key=lambda item: (item["group"], item["period"]))

        # Affichage final
        for entry in full_schedule:
            print(
                f"Groupe: {entry['group']} | "
                f"Cours: {entry['subject']} | "
                f"Enseignant: {entry['teacher']} | "
                f"{entry['period']} | "
                f"Salle: {entry['room']} | "
                f"Modalité: {entry['modality']}"
            )

        return full_schedule
    else:
        print(f"Pas de solution optimale trouvée (status = {status})")
        return None


# ------------------------------
# Exemple d'utilisation
if __name__ == "__main__":
    # group_names = les "classes" d'étudiants
    group_names = ["DIA1", "DIA2"]

    # rooms = les salles réelles
    rooms = ["A", "B", "C", "D"]

    subjects = [
        "Maths CM", "Maths TD",
        "Physique CM", "Physique TD",
        "Info CM", "Info TD"
    ]

    teachers = ["Mme Camila", "Mr Boyer", "Mme Krika"]

    # On distingue CM / TD
    course_type = {
        "Maths CM": "CM",
        "Maths TD": "TD",
        "Physique CM": "CM",
        "Physique TD": "TD",
        "Info CM": "CM",
        "Info TD": "TD"
    }

    # Pour chaque enseignant, on liste les cours qu'il sait enseigner
    teacher_specialties = {
        "Mme Camila": ["Maths CM", "Maths TD", "Physique CM", "Physique TD"],
        "Mr Boyer":   ["Maths CM", "Maths TD", "Info CM", "Info TD"],
        "Mme Krika":  ["Physique CM", "Physique TD"]
    }

    university_scheduling(
        group_names,
        rooms,
        subjects,
        teachers,
        teacher_specialties,
        course_type,
        periods_per_day=4,
        days_per_week=5
    )


Solution optimale trouvée :

Groupe: DIA1 | Cours: Maths TD | Enseignant: Mme Camila | Période 11 | Salle: D | Modalité: présentiel
Groupe: DIA1 | Cours: Info CM | Enseignant: Mr Boyer | Période 11 | Salle: Aucune salle (en ligne) | Modalité: enligne
Groupe: DIA1 | Cours: Maths CM | Enseignant: Mr Boyer | Période 13 | Salle: Aucune salle (en ligne) | Modalité: enligne
Groupe: DIA1 | Cours: Physique TD | Enseignant: Mme Camila | Période 15 | Salle: B | Modalité: présentiel
Groupe: DIA1 | Cours: Maths CM | Enseignant: Mr Boyer | Période 19 | Salle: Aucune salle (en ligne) | Modalité: enligne
Groupe: DIA1 | Cours: Physique CM | Enseignant: Mme Krika | Période 3 | Salle: Aucune salle (en ligne) | Modalité: enligne
Groupe: DIA1 | Cours: Info TD | Enseignant: Mr Boyer | Période 4 | Salle: D | Modalité: présentiel
Groupe: DIA1 | Cours: Physique TD | Enseignant: Mme Krika | Période 5 | Salle: A | Modalité: présentiel
Groupe: DIA1 | Cours: Maths TD | Enseignant: Mme Camila | Période 6 | Salle: 